In [1]:
import pandas as pd

In [2]:
TRAIN_DATASET_FILE = 'data/sales_train.csv.gz'
TEST_DATASET_FILE = 'data/test.csv.gz'
SHOP_INFO_FILE = 'data/shops.csv'
ITEM_INFO_FILE = 'data/items.csv'
ITEM_CAT_INFO_FILE = 'data/item_categories.csv'
SAMPLE_SUB_FILE ='data/sample_submission.csv.gz'

## Load of the data

In [3]:
raw_train_dataset = pd.read_csv(TRAIN_DATASET_FILE)
print(raw_train_dataset.shape)
raw_train_dataset.head()

(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


### Dealing with the missing values

In [10]:
set(raw_train_dataset[raw_train_dataset.item_cnt_day < 0].item_cnt_day.values)

{-22.0, -16.0, -9.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0}

In [13]:
raw_train_dataset["item_cnt_day"] = raw_train_dataset["item_cnt_day"].apply(lambda x: max(x, 0)) 

In [14]:
set(raw_train_dataset[raw_train_dataset.item_price <= 0].item_price.values)

{-1.0}

In [18]:
mean_price_item = raw_train_dataset[raw_train_dataset.item_price > 0].groupby(["shop_id", "item_id"],
                                                            as_index=False)["item_price"].mean()
mean_price_item_dict = {}
for shop, item, price in mean_price_item[["shop_id", "item_id", "item_price"]].values:
    if int(shop) in mean_price_item_dict.keys():
        mean_price_item_dict[int(shop)][int(item)] = price
    else:
        mean_price_item_dict[int(shop)] = {int(item): price}
mean_price_item_dict

{0: {30: 265.0,
  31: 434.0,
  32: 221.0,
  33: 347.0,
  35: 247.0,
  36: 357.0,
  40: 127.0,
  42: 127.0,
  43: 221.0,
  49: 127.0,
  51: 127.75,
  57: 167.0,
  59: 110.0,
  61: 195.0,
  75: 76.0,
  85: 190.0,
  88: 76.0,
  95: 193.0,
  96: 70.0,
  97: 67.0,
  98: 268.0,
  111: 94.0,
  115: 99.0,
  136: 89.0,
  149: 99.0,
  151: 75.0,
  152: 97.0,
  153: 258.0,
  198: 112.0,
  199: 116.0,
  210: 118.0,
  228: 75.0,
  282: 109.0,
  306: 59.5,
  308: 89.0,
  351: 89.0,
  407: 97.0,
  421: 162.0,
  438: 162.0,
  471: 227.0,
  479: 140.0,
  482: 1360.0,
  484: 124.0,
  491: 361.0,
  496: 1896.0,
  498: 124.0,
  534: 210.0,
  545: 126.0,
  556: 90.0,
  560: 99.0,
  582: 91.0,
  642: 90.0,
  651: 90.0,
  657: 90.0,
  668: 91.0,
  677: 90.0,
  687: 248.0,
  699: 143.0,
  700: 143.0,
  715: 91.0,
  783: 71.0,
  785: 155.0,
  810: 120.0,
  814: 120.0,
  816: 72.0,
  825: 195.0,
  831: 1896.0,
  835: 1896.0,
  839: 1360.0,
  944: 339.0,
  947: 37.0,
  952: 321.0,
  963: 36.0,
  965: 190.0,
  97

In [20]:
def remove_neg_price(r):
    if r.item_price > 0:
        return r
    else:
        r.item_price = mean_price_item_dict[int(r.shop_id)][int(r.item_id)]
        return r
        
raw_train_dataset_without_missing_values = raw_train_dataset.apply(remove_neg_price, axis=1)

In [22]:
raw_train_dataset_without_missing_values.to_csv("data/raw_train_dataset_without_missing_values.csv.gz",
                                                compression='gzip')

## First we will group the raw by month

we will use the sum of the number of sales, and the mean et the std deviation for the price

In [ ]:
raw_train_dataset["mean_price"] = raw_train_dataset["price"]
raw_rain_dataset["std_price"] = raw_train_dataset["price"]
grouped_train_data = raw_rain_dataset.groupby(["date_block_num", "shop_id", "item_id"],
                                              as_index=False).agg({"mean_price": "avg", "std_price": "std", ""})
grouped_train_data = 